In [6]:
# Libraries
import os
import json
import pandas as pd

In [7]:
class gallovidia:
    def __init__(self):
        self.path = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/"
    def record_transaction(self):
        """
        Users can record transactions.
        1. State path.
        2. Enter transaction details.
        3. Write transaction to JSON; dump in folder.
        """
        # 1. State path.
        path = self.path
        
        # 2. Enter transaction details.
        """
        Flow: Money added (e.g. deposit, dividend, sale) and money removed (e.g. purchase, withdrawal).
            - Internal transactions will affect Flow transactions (e.g. selling $MSFT adds money to Flow account).
        Internal: Any purchase (takes money from Flow account) or sale (adds money to Flow account) of securities.
        """
        def transaction_details():
            while True:
                print("Transactions can vary./nFLow: Money going into or out of the portfolio./nInternal: No new money, just transactions.")
                t = input("Do you wish to record a flow or internal transaction?")
                
                if t == 'internal' or t == 'Internal' or t == 'Internal transaction' or t == 'internal transaction':
                    internal = {
                        "date": input("Enter transaction date (YYYY-MM-DD): "),
                        "account": input("Enter account name: "),
                        "security": input("Enter security name: "),
                        "ticker": input("Enter ticker symbol (e.g. MSFT for Microsoft)"),
                        "exchange country": input("Enter country of exchange"),
                        "value": input("Enter value of transaction: "),
                        "quantity": input("Enter quantity: "),
                        "price": input("Enter price: "),
                        "type": input("Enter transaction type (buy/sell/dividend): "),
                        "transaction_group": "internal",
                        "currency_ISO": input("Enter the currency ISO"),
                        "fx_rate": input("Enter FX rate (if applicable): "),
                        "fees": input("Enter fees (if applicable): ")
                        }
                    return internal
                elif t == 'flow' or t == 'Flow' or t == 'flow transaction' or t == 'Flow transaction':
                    flow = {
                        "date": input("Enter transaction date (YYYY-MM-DD): "),
                        "account": input("Enter account name: "),
                        "value": input("Enter value of transaction: "),
                        "type": input("Enter transaction type (deposit/dividend/sale): "),
                        "transaction_group": "flow",
                        "currency_ISO": input("Enter the currency ISO"),
                        "fx_rate": input("Enter FX rate (if applicable): "),
                        "fees": input("Enter fees (if applicable): ")
                        }
                    if flow['type'] == 'dividend':
                        # add dividend information
                        flow['security'] = input("Enter security name: ")
                        flow['ticker'] = input("Enter ticker symbol (e.g. MSFT for Microsoft)")
                        flow['exchange country'] =input("Enter country of exchange")
                    return flow
                else:
                    print("Invalid response. Please try again.")
                    continue
        
        transaction = transaction_details()
        # 3. Validate transaction
        """
        Give users the option to ammend any part of the dictionary.
        """
        while True:
            print(transaction)
            print("/nOptions:/n-Edit/n-New field/n-Continue")
            x = input("Would you like to make any changes to the transaction?")
            if  x == "Continue":
                break
            # edit a certain part of the code
            elif x == "Edit":  
                editfield = input(f"Which field recorded would you like to edit?/n/n{transaction.keys()}")
                transaction[editfield] = input("Enter new value.")
                print("Transaction changed.")
                continue
            # add a new field to the transaction
            elif x == "New field":
                newfield = input(f"What new field would you like to add?/n/n{transaction.keys()}")
                transaction[newfield] = input("Enter new value.")
                print("Transaction changed.")
            else:
                print("Invalid response. Please try again.")
                continue
        
        # 4. Write transaction to JSON; dump in folder.
        if transaction['transaction_group'] == 'flow':
            with open(f"{path}{transaction['date']}{transaction['transaction_group']}{transaction['type']}.json", "w") as file:
                json.dump(transaction, file)
        elif transaction['transaction_group'] == 'internal':
            with open(f"{path}{transaction['date']}{transaction['transaction_group']}{transaction['type']}{transaction['ticker']}.json", "w") as file:
                json.dump(transaction, file)
    def create_excel(self):
        """
        Take all JSON files saved in transactions folder and make them into an excel sheet.
        - 'Export' functionality; 'Edit' functionality.

        N.B.    The reason why I am taking this approach, JSON -> Excel (.csv), is that 
                I want users to be able to create new fields, but keep legacy transactions valid
                e.g. client, SEDOL, dealer, etc.
        """
        path = self.path
        json_list = []

        for filename in os.listdir(path):
            if filename.endswith('.json'):
                with open(os.path.join(path, filename)) as f:
                    data = json.load(f)
                    json_list.append(data)
        
        return pd.DataFrame.from_dict(json_list)
    def import_transaction(self, path):
        """
        Import a transaction from an excel or csv sheet.
        :param path: path to excel sheet
        """
        # Read excel sheet or csv file, convert to individual JSON files.
        if path.endswith('.csv'):
            df = pd.read_csv(path)
        elif path.endswith('.xlsx'):
            df = pd.read_excel(path)
        else:
            raise ValueError("Invalid file type. Please try again.")

        for i in range(len(df)):
            with open(f"{self.path}{df['date'][i]}{df['transaction_group'][i]}{df['type'][i]}.json", "w") as file:
                json.dump(df.iloc[i].to_dict(), file)
        print("Transactions imported.")



In [ ]:
from django.shortcuts import render, redirect
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_protect
from django.models import Transaction

@csrf_protect
def record_transaction(request):
    if request.method == "POST":
        # Get the data that the user has entered into the form.
        date = request.POST["date"]
        account = request.POST["account"]
        value = request.POST["value"]
        type = request.POST["type"]
        transaction_group = request.POST["transaction_group"]
        currency_ISO = request.POST["currency_ISO"]
        fx_rate = request.POST["fx_rate"]
        fees = request.POST["fees"]

        # Create a new transaction object.
        transaction = Transaction(
            date=date,
            account=account,
            value=value,
            type=type,
            transaction_group=transaction_group,
            currency_ISO=currency_ISO,
            fx_rate=fx_rate,
            fees=fees
        )

        # Save the transaction object to the database.
        transaction.save()

        # Redirect the user to a confirmation page.
        return redirect("confirmation")

    else:
        return render(request, "record_transaction.html")


In [ ]:
record_transaction("POST")

In [ ]:
def import_transaction(self, path):
    """
    Import a transaction from an excel or csv sheet.
    :param path: path to excel sheet
    """ 
    # Identify from the filepath if the file is from AJBell, Interactive Investors, or FreeTrade.
    if path.contains("AJBell"):
        source = "AJBell"
    elif path.contains("Interactive Investors"):
        source = "Interactive Investors"
    elif path.contains("FreeTrade"):
        source = "FreeTrade"
    else:
        raise ValueError("Invalid file type. Please try again.")
    
    # Identify if the account type of the transaction
    if path.contains("ISA"):
        account_type = "ISA"
    elif path.contains("SIPP"):
        account_type = "SIPP"
    elif path.contains("LISA"):
        account_type = "LISA"
    elif path.contains("GIA"):
        account_type = "GIA"
    else:
        raise ValueError("Invalid file type. Please try again.")
    
    # Read excel sheet or csv file, convert to individual JSON files.
    if path.endswith('.csv'):
        df = pd.read_csv(path)
    elif path.endswith('.xlsx'):
        df = pd.read_excel(path)
    else:
        raise ValueError("Invalid file type. Please try again.")

    for i in range(len(df)):
        


        with open(f"{self.path}{df['date'][i]}{df['transaction_group'][i]}{df['type'][i]}.json", "w") as file:
            json.dump(df.iloc[i].to_dict(), file)
    print("Transactions imported.")


In [ ]:
# Record a transaction
gallovidia().record_transaction()

In [8]:
# View transactions
gallovidia().create_excel()

,date,account,security,ticker,exchange country,value,quantity,price,type,currency_ISO,fx_rate,fees,transaction_group
0,2023-04-05,AJBell_ISA,ASML,ASML,Netherlands,1086.49,2,,buy,EUR,,,NaN
1,2023-04-05,AJBell_ISA,Microsoft,MSFT,USA,1627.04,7,,buy,USD,,,NaN
2,2023-04-05,AJBell_ISA,SSE,SSE,United Kingdom,1014.76,55,,buy,GBp,,,NaN
3,2023-04-05,AJBell_ISA,Texas Instruments,TXN,USA,1167.2,8,,buy,USD,,,internal
4,2023-05-02,AJBell_LISA,NaN,NaN,NaN,4000,NaN,NaN,deposit,GBP,,,flow
5,2023-05-09,Freetrade_ISA,NaN,NaN,NaN,400,NaN,NaN,deposit,GBP,,,flow
6,2023-05-09,Freetrade_ISA,PayPal,PYPL,USA,379.42,7.09347048,67.07852,buy,USD,1.2615,2.23,internal


In [19]:
# Import transactions
#gallovidia().import_transaction(path = 'C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/transactionhistory, AJBell_ISA, 2017.csv')
pd.read_csv('C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/transactionhistory, AJBell_ISA, 2017.csv')

,Date,Transaction,Description,Quantity,Price,Amount (GBP),Reference
0,15/12/2017,Purchase,Visa Inc Class A,4.0000,£84.7654,352.41,44617V11486
1,15/12/2017,Sale,Oracle Corp,10.0000,£35.4663,341.16,44617V11097
2,04/12/2017,Purchase,ING Groep NV,29.0000,£13.3194,400.08,44617U13586
3,15/11/2017,Purchase,Block Inc Class A,16.0000,£30.5444,503.55,44617T10495
4,06/11/2017,Purchase,Intel Corp,13.0000,£35.4685,475.66,44617S21432
5,06/11/2017,Purchase,NORDEA BANK AB NPV,43.0000,£9.0817,404.37,44617S20689
6,27/10/2017,Purchase,Oracle Corp,10.0000,£38.3503,397.29,44617R77257
7,25/10/2017,Purchase,PayPal Holdings Inc,10.0000,£53.2125,547.41,44617R67466
8,18/10/2017,Purchase,JUST EAT PLC ORD GBP0.01,40.0000,£7.16895,298.14,44617R35665
9,29/09/2017,Equalisation Acc Units,Jupiter India I Acc,0.0000,NaN,0.66,44617##0009


In [17]:
# Import transactions from excel / csv
# Loop through files in folder: C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions
filepath = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/"
for i in os.listdir(filepath):
    if i.endswith('.csv') or i.endswith('.xlsx'):
        print(filepath + i)
        gallovidia().import_transaction(filepath + i)

C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/transactionhistory, AJBell_ISA, 2017.csv


KeyError: 'date'

In [ ]:
def importfrom(path, source, delete = False):

    if source == 'AJ Bell' or source == 'AJBell':
        files = os.listdir(path = path)
        files = [os.path.join(path, f) for f in files
                 if 'AJBell' in f and f.endswith('.csv')]
        df = pd.concat([pd.read_csv(f)
                        .assign(account_type = 'AJBell_LISA' if 'AJBell_LISA' in f else 'AJBell_ISA') 
                        for f in files])
        
        return df

In [ ]:
df = importfrom(path = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions",
           source = 'AJ Bell')

In [ ]:
for index, rows in df.iterrows():
    filename = os.path.join(path = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions",
                            f"{rows['filename']}.json")

In [ ]:

#gallovidia().create_excel()